In [0]:
/*
create table hive_metastore.default.cm_4g_cell_records_bk_20241118
 select * from hive_metastore.default.cm_4g_cell_records;

select count(*) from hive_metastore.default.cm_4g_cell_records_bk_20241118;  -- 86086877
select count(*) from hive_metastore.default.cm_4g_cell_records;   -- 86086877  84756044

select * 
-- delete
from hive_metastore.default.cm_4g_cell_records where Date >= '2024-11-14' and Date <= '2024-11-16';

*/


delete from hive_metastore.default.cm_4g_cell_records where date=CURRENT_DATE()-1;


INSERT INTO hive_metastore.default.cm_4g_cell_records
select Date,ne_id, cnum,frequency_band_indicator,
case  when (frequency_band_indicator) = 2 then "1900mhz"
when (frequency_band_indicator) = 4 then "2100mhz_band4"
when (frequency_band_indicator) = 5 then "850mhz"
when (frequency_band_indicator) = 13 then "700mhz"
when (frequency_band_indicator) = 46 then "5000mhz"
when (frequency_band_indicator) = 48 then "3500mhz"
when (frequency_band_indicator) = 66 then "2100mhz_band66"
end as  BAND_NAME
from (
  SELECT current_date()-1 as Date,ne_id, cnum, min(frequency_band_indicator) frequency_band_indicator
  FROM read_files(
    's3://apacheupsertdata/cm_data_config/cm_4g_cell_records_{{date}}.csv',
    format => 'csv',
    header => true)
    where frequency_band_indicator is not null
  GROUP BY ne_id, cnum
) x
;


TRUNCATE TABLE `hive_metastore`.`default`.`vz_conf_ecell_silver_tmp`;


/*
insert into `hive_metastore`.`default`.`vz_conf_ecell_silver_tmp`
(ENODEB_ID,
ECELL_ID,
MARKET_NAME,
SUB_MARKET_ID,
SUB_MARKET_NAME,
BAND_NAME
)
select distinct replace(a.NE_ID, 'eNB', 'VZ') AS ENODEB_ID,
cast(replace(a.LOCATION, 'cNum', '') as bigint) AS ECELL_ID,
d.REGION AS MARKET_NAME,
a.SITE_ID  AS SUB_MARKET_ID,
d.SUB_MARKET_NAME,
c.BAND_NAME
FROM apachedataingestion.lte.availability_lte_hr_silver a
left join hive_metastore.default.vz_conf_market d on a.SITE_ID = d.SUB_MARKET_ID
left join (
          select a.*
          from hive_metastore.default.cm_4g_cell_records   a
          inner join (
                    select 
                      Max(Date) As Date,ENODEB_ID, ECELL_ID  
                    from hive_metastore.default.cm_4g_cell_records 
                    where Date>=(CURRENT_DATE - INTERVAL 4 DAY)  
                    GROUP BY ENODEB_ID, ECELL_ID
              ) b on a.ENODEB_ID = b.ENODEB_ID and a.ECELL_ID = b.ECELL_ID and a.date = b.date 
) c on lower(a.NE_ID) = lower(c.ENODEB_ID) and cast(replace(a.LOCATION, 'cNum', '') as bigint) = c.ECELL_ID
where a.Daily>=(CURRENT_DATE - INTERVAL 4 DAY);



ALTER TABLE `hive_metastore`.`default`.`vz_conf_ecell_silver_tmp` ADD COLUMN oNE_ID String;
ALTER TABLE `hive_metastore`.`default`.`vz_conf_ecell_silver_tmp` ADD COLUMN oNE_NAME String;

*/



insert into `hive_metastore`.`default`.`vz_conf_ecell_silver_tmp`
(
--oNE_ID,
--oNE_NAME,
ENODEB_ID,
ECELL_ID,
MARKET_NAME,
SUB_MARKET_ID,
SUB_MARKET_NAME,
BAND_NAME
)
select distinct 
--a.NE_ID oNE_ID,a.NE_NAME oNE_NAME, -- added
replace(a.NE_ID, 'eNB', 'VZ') AS ENODEB_ID,
cast(replace(a.LOCATION, 'cNum', '') as bigint) AS ECELL_ID,
d.REGION AS MARKET_NAME,
a.SITE_ID  AS SUB_MARKET_ID,
d.SUB_MARKET_NAME,
c.BAND_NAME
FROM apachedataingestion.lte.availability_lte_hr_silver a
left join hive_metastore.default.vz_conf_market d on a.SITE_ID = d.SUB_MARKET_ID
left join (
select a.*
from hive_metastore.default.cm_4g_cell_records   a
inner join (select Max(Date) As Date,ENODEB_ID, ECELL_ID
from hive_metastore.default.cm_4g_cell_records
GROUP BY ENODEB_ID, ECELL_ID) b on a.ENODEB_ID = b.ENODEB_ID and a.ECELL_ID = b.ECELL_ID and a.date = b.date 
) c on lower(a.NE_ID) = lower(c.ENODEB_ID) and cast(replace(a.LOCATION, 'cNum', '') as bigint) = c.ECELL_ID
where a.Daily>=(CURRENT_DATE - INTERVAL 2 DAY);


--merge records 
MERGE INTO hive_metastore.default.vz_conf_ecell_silver AS a
USING hive_metastore.default.vz_conf_ecell_silver_tmp AS b
ON a.ENODEB_ID = b.ENODEB_ID AND a.ECELL_ID = b.ECELL_ID
WHEN MATCHED THEN
  UPDATE SET a.ENODEB_ID = b.ENODEB_ID,
             a.ECELL_ID = b.ECELL_ID,
             a.MARKET_NAME = b.MARKET_NAME,
             a.SUB_MARKET_ID = b.SUB_MARKET_ID,
             a.SUB_MARKET_NAME = b.SUB_MARKET_NAME,
             a.BAND_NAME = b.BAND_NAME;


--insert records
insert into hive_metastore.default.vz_conf_ecell_silver
(ENODEB_ID,
ECELL_ID,
MARKET_NAME,
SUB_MARKET_ID,
SUB_MARKET_NAME,
BAND_NAME
)
select 
a.ENODEB_ID,
a.ECELL_ID,
a.MARKET_NAME,
a.SUB_MARKET_ID,
a.SUB_MARKET_NAME,
a.BAND_NAME
from hive_metastore.default.vz_conf_ecell_silver_tmp a
left join hive_metastore.default.vz_conf_ecell_silver b
on a.ENODEB_ID = b.ENODEB_ID AND a.ECELL_ID = b.ECELL_ID
WHERE b.ENODEB_ID IS NULL;

-- select * from hive_metastore.default.vz_conf_ecell_silver where ENODEB_ID='UADPF_11093002239' and ECELL_ID='27'

